In [ ]:
!pip install sentencepiece
!pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pla

In [ ]:
from datasets import load_dataset
from itertools import islice

num_samples = 50000
output_txt_path = "/content/c4_subset.txt"

print("[Dataset] Streaming C4...")
stream = load_dataset("c4", "en", split="train", streaming=True)
stream = list(islice(stream, num_samples))

print(f"[Dataset] Writing {num_samples} samples to {output_txt_path}")
with open(output_txt_path, "w", encoding="utf-8") as f:
    for ex in stream:
        f.write(ex["text"].replace("\n", " ") + "\n")  # Normalize newlines


[Dataset] Streaming C4...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.21k [00:00<?, ?B/s]

c4.py:   0%|          | 0.00/3.46k [00:00<?, ?B/s]

The repository for c4 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/c4.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


/root/.cache/huggingface/modules/datasets_modules/datasets/c4/584d57ebe81c209b6c7f31727066d2c4b4bba37cb7092cdd83083d5ec11207db/c4.py:53: FutureWarning: Dataset 'c4' is deprecated and will be deleted. Use 'allenai/c4' instead.
  warnings.warn(


[Dataset] Writing 50000 samples to /content/c4_subset.txt


In [ ]:
import sentencepiece as spm

model_prefix = "c4_spm_8000_final"
spm.SentencePieceTrainer.train(
    input=output_txt_path,
    model_prefix=model_prefix,
    vocab_size=8000,
    character_coverage=1.0,
    model_type="unigram",
    pad_id=0,
    unk_id=1,
    bos_id=2,
    eos_id=3,
    # REMOVE user_defined_symbols
)

# This creates two files:
# - c4_spm_8000.model
# - c4_spm_8000.vocab


In [ ]:
import sentencepiece as spm
import torch

class SentencePieceTokenizer:
    def __init__(self, model_path, max_length=1024):
        self.sp = spm.SentencePieceProcessor()
        self.sp.load(model_path)
        self.max_length = max_length
        self.bos_id = self.sp.bos_id()
        self.eos_id = self.sp.eos_id()
        self.unk_id = self.sp.unk_id()
        self.pad_id = self.sp.pad_id()

    def __call__(self, texts, truncation=True, padding="max_length", return_tensors="pt"):
        token_ids = []
        for text in texts:
            ids = self.sp.encode(text, out_type=int)
            if truncation:
                ids = ids[:self.max_length]
            if padding == "max_length":
                ids += [self.pad_id] * (self.max_length - len(ids))
            token_ids.append(ids)
        return {"input_ids": torch.tensor(token_ids)}


In [ ]:
tokenizer = SentencePieceTokenizer("/content/drive/MyDrive/c4_spm_8000_final.model")
example = "The quick brown fox jumps over the lazy dog."
print("Tokens:", tokenizer.sp.encode(example, out_type=str))
print("IDs:", tokenizer.sp.encode(example, out_type=int))


Tokens: ['▁The', '▁quick', '▁brown', '▁fo', 'x', '▁jump', 's', '▁over', '▁the', '▁la', 'zy', '▁dog', '.']
IDs: [28, 1817, 3545, 1133, 240, 3290, 4, 134, 7, 658, 3016, 1508, 5]


In [ ]:
tokenizer.pad_id

0

In [ ]:
tokenizer.sp.id_to_piece(0)

'<pad>'